# Preparation of Flux-Free Kitaev Honeycomb lattice state
## In HONEYCOMB LATTICE SPACE
2 honeycombs

N = 14 spins

Hamiltonian: $H = -J_x \sum_{\langle i,j \rangle_x} \sigma_i^x \sigma_j^x
    -J_y \sum_{\langle i,j \rangle_y} \sigma_i^y \sigma_j^y
    -J_z \sum_{\langle i,j \rangle_z} \sigma_i^z \sigma_j^z$ 

Step 1: prepare product state $|00..>$ of N spins

In [2]:
import numpy as np
import scipy
from scipy import sparse

KeyboardInterrupt: 

We are doing this mapping: 

$\ket{\uparrow \uparrow} = \ket{00} , \quad\ket{\downarrow \downarrow} = \ket{11} $


![Graphene]()



In [2]:
n_spins = 14
e1 = sparse.csr_array([[1],[0]])
psi = e1
for _ in range(n_spins-1):
    psi = sparse.kron(psi,e1, format='csr')

print(psi.shape)


assert psi.shape[0]== 2**14

(16384, 1)


In [3]:
print(psi)

  (0, 0)	1


Identify a representative qubit in each plaquette: we identify qubit 0 and 6

We create operator that applies hadamard only on these two representative qubits

In [5]:
def Hadamard(qubit1, qubit2, n):
    assert qubit1 < qubit2
    Id = sparse.csr_array(np.eye(4))
    H_small = sparse.csr_array([[1./np.sqrt(2), 0., 0., 1./np.sqrt(2)], [0., 1., 0., 0.], [0., 0., 1., 0.], [1./np.sqrt(2), 0., 0., -1./np.sqrt(2)]])
    op_list = [Id]*n
    op_list[qubit1] = H_small
    op_list[qubit2] = H_small
    
    H = op_list[0]

    for op in op_list[1:]:
        H = sparse.kron(H,op, format='csr')
    return H
    

In [ ]:
qubit1 = 0
qubit2 = 4
H = Hadamard(qubit1,qubit2,n_spins)
print(H.shape)
psi1 = H @ psi.copy()

print(psi1.shape)
print(psi1)

: 

Create op. that applies CNOT on each plaquette with representative qubits as control qubits and all other qubits of plaquette as target ones

In [1]:
def CNOT(qubit1, qubit2, n, l): #l = spins per plaquette, n = tot spins
    assert qubit1 < 3
    assert 3 < qubit2
    # Id = sparse.csr_array(np.eye(2))
    # X = sparse.csr_array([[0.,1.],[1.,0.]])
    # Proj_0 = sparse.csr_array([[1.,0.],[0.,0.]]) #|0><0|
    # Proj_1 = sparse.csr_array([[0.,0.],[0.,1.]]) #|1><1|
    
    newId = sparse.csr_array(np.eye(2**(n-2*l)))
    CNOT_p1 = sparse.kron(plaquetteCNOT(qubit1,l),newId,'csr')
    CNOT_p2 = sparse.kron(newId,plaquetteCNOT(qubit2-l+1,l), 'csr') #we map qubit 3 -> 0, 4->1, 5->2, 6->3
    # even if geometry is not conserved, it still works because CNOT applied on this 
    #sequence of hilbert spaces in this order does not depend on geometry!
    # print(CNOT_p2 @ CNOT_p1)
    # print(CNOT_p1 @ CNOT_p2)
    assert np.array_equal((CNOT_p2 @ CNOT_p1).toarray(), (CNOT_p1 @ CNOT_p2).toarray()), "Sparse arrays are not equal"
    return CNOT_p2 @ CNOT_p1

def plaquetteCNOT(qubit, l):
    Id = sparse.csr_array(np.eye(4))
    X = np.zeros((4,4))
    X[0,3] = 1
    X[3,0] = 1
    X = sparse.csr_array(X)
    print(X)
    
    Proj_0 = np.zeros((4,4)) #|00><00|
    Proj_0[0,0] = 1
    Proj_0 = sparse.csr_array(Proj_0)
    Proj_1 = np.zeros((4,4)) #|11><11|
    Proj_1[3,3] = 1
    Proj_1 = sparse.csr_array(Proj_1)

    op_list_id = [Id]*l
    op_list_id[qubit] = Proj_0

    op_list_x = [X]*l
    op_list_x[qubit] = Proj_1

    CNOT1 = op_list_id[0]
    CNOT2 = op_list_x[0]

    for op1, op2 in zip(op_list_id[1:],op_list_x[1:]):
        CNOT1 = sparse.kron(CNOT1, op1, 'csr')
        CNOT2 = sparse.kron(CNOT2, op2, 'csr')
    return CNOT1 + CNOT2

In [11]:
#TEST FOR plaquetteCNOT: it works
CNOT_test = plaquetteCNOT(0,2)
print(CNOT_test*([1,0,1,0]))

  (0, 0)	1.0
  (1, 1)	0.0
  (2, 3)	0.0
  (3, 2)	1.0


In [ ]:
CNOT_op = CNOT(qubit1, qubit2, n_spins, 4)
#print(CNOT_op.toarray())
psi2 =  CNOT_op @ psi1.copy()
#print(psi1.shape)
#print(CNOT_op.shape)
#print(psi2.shape)
print(psi2)

[0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.
 0.  0. ]


Now we want to convert this state from toric code space to Honeycomb space! Mapping written on ipad

In [31]:
# Example 1D sparse array
n = n_spins  # Length of binary representation

sparse_array = psi2.copy()

# Step 1: Extract non-zero positions
non_zero_positions = sparse_array.nonzero()[0]
print(non_zero_positions)

# Step 2: Convert positions to binary strings of length n
binary_positions = [f"{pos:0{n}b}" for pos in non_zero_positions]

# Step 3: Extract positions of '1's and compute new positions
new_binary_positions = []

for binary in binary_positions:
    ones_positions = [i for i, bit in enumerate(binary) if bit == '1']
    new_binary = ['0'] * (2 * n)

    for pos in ones_positions:
        new_binary[2 * pos] = '1'  # Place '1' at 2 * pos
        if 2 * pos + 1 < len(new_binary):  # Ensure within bounds
            new_binary[2 * pos + 1] = '1'  # Place '1' at 2 * pos + 1

    new_binary_positions.append(''.join(new_binary))

# Step 4: Convert new binary numbers to integers
new_positions = [int(binary, 2) for binary in new_binary_positions]

# Step 5: Map original data values to new positions
data = sparse_array.data  # Extract the non-zero values from the original sparse array
print(data)

# Step 6: Create new sparse array
new_sparse_array = sparse.csr_array((data, (new_positions, [0] * len(new_positions))), shape=(2**(2*n), 1))

# Print results
print("Original Sparse Array:")
print(sparse_array)

print("\nNew Sparse Array:")
print(new_sparse_array)

[  0  15 119 120]
[0.5 0.5 0.5 0.5]
Original Sparse Array:
  (0, 0)	0.4999999999999999
  (15, 0)	0.4999999999999999
  (119, 0)	0.4999999999999999
  (120, 0)	0.4999999999999999

New Sparse Array:
  (0, 0)	0.4999999999999999
  (255, 0)	0.4999999999999999
  (16191, 0)	0.4999999999999999
  (16320, 0)	0.4999999999999999


16384
